In [1]:
import torch
from PIL import Image
from gsplat.rendering import rasterization

In [10]:
means = torch.rand((100, 3))
quats = torch.rand((100, 4))
quats = quats / torch.linalg.norm(quats, dim=1, keepdim=True)
scales = torch.rand((100, 3))
colors = torch.rand((100, 3))
opacities = torch.rand((100, 1))

In [ ]:
render_colors, render_alphas, info = rasterization(
            means=means,
            quats=quats,
            scales=scales,
            opacities=opacities,
            colors=colors,
            viewmats=torch.linalg.inv(camtoworlds),  # [C, 4, 4]
            Ks=Ks,  # [C, 3, 3]
            width=width,
            height=height,
            packed=self.cfg.packed,
            absgrad=self.cfg.absgrad,
            sparse_grad=self.cfg.sparse_grad,
            rasterize_mode=rasterize_mode,
            **kwargs,
        )